In [344]:

from nltk.corpus import wordnet as wn
import numpy as np
import pandas as pd
import wikipedia
from urllib.request import urlopen
import json
api = "https://www.googleapis.com/books/v1/volumes?q=title:"
isbnAPI = "https://www.googleapis.com/books/v1/volumes?q=isbn:"


## Big idea- Using wikipedia pages to infer ethnicity
Keywords and permulations
- taking the associated catagories that are used, and using nltk's wordnet to find all associated words
Searching
- opening up the author's wikipedia article, and searching for the frequencies of all the keywords. At the end of the search, whichever catagory has the most mentions, is the catagory the author is sorted into
- if there are no mentions, check the author's birthplace and parse that
- if no birthplace, people assume whiteness inherently when writting, so the same must be done. That OR the prediction thing

In [345]:

asian = np.asarray(['china','japan','korea','taiwan','east-asian','japanese','korean','chinese','taiwanese','india','pakistan','bangledesh','nepal','south-asian','hindi','urdu'])
black = np.asarray(['african-american','nigera','ethiopia','egypt','african','nigerian','ethiopian','egyptian'])
# black != african, but african is a subset of black.
white = np.asarray(['european','caucasian'])
hispanic = np.asarray(['hispanic','mexico','spanish','Latino','Latin'])
keywords = np.asarray([asian,black,white,hispanic])
keywordTitles = ['asian','black','white','hispanic']
for j in range(len(keywords)):
    for i in range(len(keywords[j])):
        for ss in wn.synsets(keywords[j][i]):
            keywords[j] = np.append(keywords[j],np.asarray(ss.lemma_names()),axis = None)
    keywords[j].flatten()
    keywords[j] = list(np.unique(keywords[j]))

keywordDF = (pd.DataFrame(keywords).T)

keywordDF.columns =keywordTitles
# keywordDF.set_index(['asian']).apply(pd.Series.explode).reset_index()

C:\Users\amer_\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [346]:
def getBookInfo(title):
    title = title.replace(' ', '%')
    url = api + title
    response = urlopen(url)
    data = json.loads(response.read())
    return data

In [351]:
title = ['How to Invent Everything','Percy Jackson','Oxford English Dictionary','The Fault in Our Stars','The Alchemist','The Hunger Games','To Kill a Mockingbird','The Kite Runner','0994047134','Fifteen Hundred Miles from the Sun']
namesDF = pd.DataFrame()
for bookTitle in title:
    bookInfo = getBookInfo(bookTitle)['items'][0]['volumeInfo']
    author = bookInfo['authors'][0].split(' ')
    lastName = author[-1]
    firstName = author[0]
    print(author)
    name = firstName + ' ' + lastName
    # Taking the first result for the search query which is index 0, getting information about the book and then printing the author of the book.
    namesDF = namesDF.append({'Title':bookInfo['title'],'author' : name, 'last':lastName,'first': firstName},ignore_index=True)
namesDF

['Ryan', 'North']
['Rick', 'Riordan']
['John', 'Andrew', 'Simpson']
['John', 'Green']
['Paulo', 'Coelho']
['Suzanne', 'Collins']
['Harper', 'Lee']
['Khaled', 'Hosseini']
['Kai', 'Cheng', 'Thom']
['Jonny', 'Garza', 'Villa']


,Title,author,last,first
0,How to Invent Everything,Ryan North,North,Ryan
1,"Lightning Thief, The (Percy Jackson and the Ol...",Rick Riordan,Riordan,Rick
2,The Oxford English Dictionary,John Simpson,Simpson,John
3,The Fault in Our Stars,John Green,Green,John
4,The Alchemist,Paulo Coelho,Coelho,Paulo
5,The Ballad of Songbirds and Snakes (A Hunger G...,Suzanne Collins,Collins,Suzanne
6,To Kill a Mockingbird,Harper Lee,Lee,Harper
7,The Kite Runner,Khaled Hosseini,Hosseini,Khaled
8,Fierce Femmes and Notorious Liars,Kai Thom,Thom,Kai
9,Fifteen Hundred Miles from the Sun,Jonny Villa,Villa,Jonny


In [353]:
for i,author in enumerate(namesDF['author']):
    bookTitle= namesDF['Title'][i]
    # author = 
    try:
        page_object = wikipedia.page(wikipedia.search(author)[0])
    except Exception as inst:
        try:
            page_object = wikipedia.page(author + ' (author)')
        except Exception as inst:
            searches = wikipedia.search(author)[1:]
            for search in (searches):
                try:
                    page_object = wikipedia.page(search)
                    if(page_object.content.find(bookTitle) != -1):
                        # checks if the page is about the author
                        break
                except Exception as inst:
                    print('No author found')
                    page_object = 'nan'
                    continue
    print(page_object.title)


Ryan North
Rick Riordan


C:\Users\amer_\AppData\Local\Programs\Python\Python39\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\amer_\AppData\Local\Programs\Python\Python39\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


No author found
<built-in method title of str object at 0x000001D53E7E9DF0>
John Green (author)
Paulo Coelho
Suzanne Collins
Harper Lee
Khaled Hosseini
Kai Cheng Thom
Dean Smith (footballer, born 1971)


In [349]:
wikipedia.search('Jonny Villa')

['Jonny Evans',
 'Jonny Gould',
 'Jonny Howson',
 'Love Island (2015 TV series, series 3)',
 'Jonny Bogris',
 'Viva Villa!',
 'Jonny Z',
 'Kaine Kesler Hayden',
 'Jonny Hurst',
 'Dean Smith (footballer, born 1971)']

In [350]:
title = '9781417732470'
url = isbnAPI + title
response = urlopen(url)
data = json.loads(response.read())
data['items'][0]['volumeInfo']['authors']
# if the isbn lookup doesnt work, then try the title lookup
# Some authors don't have a wikipedia page, so we need to search for them

['Rick Riordan']